In [35]:
import pandas as pd
import tqdm

import re
import requests
from bs4 import BeautifulSoup

import hashlib
import datetime

In [36]:
base_url = 'http://www.curitiba.pr.gov.br/dadosabertos/consulta'

In [37]:
params = {"grupo":"6"}

In [38]:
r = requests.get(base_url, params=params)

In [39]:
page = BeautifulSoup(r.text, 'html.parser')

In [40]:
__EVENTVALIDATION = page.find("input", {'name': '__EVENTVALIDATION'}).get('value')

In [41]:
__EVENTVALIDATION

'/wEdAAyYl2S6HDpGvRuNDj3CpFyB4mKL5o1QRo/FpCKPmYsWJJ6M545mi+eVxYOsufjJuPs5znhzSWYfXU4ugll/AJGuS7DlCF7QoIFOF8dSWE2wXzkiLQS8NWTPNZDV80Z3AzzphndoyDFnIVED7A3KQw9cs48DT8q4XVH83r8Yw95EbJ8/8wAqn890Z+wGGuGKtYqNvjaF5GeBQ6wPsZcNPGG+vCOATgFUUjl7PbiZupqgZarqqZvleoeig3SW/qIZQc2VVgq8PeUc/LZYZGEW+0jICJhbQQ=='

In [42]:
__VIEWSTATE = page.find("input", {'name': '__VIEWSTATE'}).get('value')

In [43]:
__VIEWSTATE

'...'

In [44]:
pagination = page.find_all(class_='paginacao2')
pagination = list(pagination[0].text.replace('\n', ''))

In [45]:
pages = ""
for page in pagination:
    data = {
        "__EVENTTARGET": f"ctl00$cphMasterPortal$ucPaginador${page}_pg{page}",
        "__EVENTVALIDATION": __EVENTVALIDATION,
        "__VIEWSTATE": __VIEWSTATE
    }
    
    r = requests.request("POST", base_url, params=params, data=data)
    pages = pages + r.text

In [46]:
md5 = lambda s : hashlib.md5(s.encode('utf-8')).hexdigest()

In [47]:
md5('test')

'098f6bcd4621d373cade4e832627b4f6'

In [48]:
a_tags = BeautifulSoup(pages, 'html.parser').find_all(id='linkArquivo')

In [49]:
print(md5(a_tags[0].text))

2d1c91001f68cea6a00114f40f12a5a0


In [50]:
date_regex = re.compile('\d{4}-\d{2}-\d{2}')

In [51]:
date_string = date_regex.search('//multimidia.curitiba.pr.gov.br/dadosabertos/SESPAMedicoUnidadesMunicipaisDeSaude/2018-10-10_Sistema_E-Saude_Medicos_-_Base_de_Dados.csv').group()
date = datetime.date(*map(int, date_string.split('-')))
date


datetime.date(2018, 10, 10)

In [52]:
databases = []
for a_tag in tqdm.tqdm(a_tags):
    database_link = a_tag.get('href')[2:]
    
    if database_link[-4:] == '.csv':
        database_id = md5(a_tag.text)
        database_title = a_tag.text

        database_date_string = date_regex.search(database_link).group()
        database_date = datetime.date(*map(int, database_date_string.split('-')))

        database = {
            'database_id': database_id,
            'database_title': database_title,
            'database_link': database_link,
            'database_date': database_date
        }
        
        databases.append(database)

100%|██████████| 46/46 [00:00<00:00, 39552.68it/s]


In [53]:
databases

[{'database_id': '2d1c91001f68cea6a00114f40f12a5a0',
  'database_title': 'Oficina de Musica - Categoria Nucleo - Base de Dados',
  'database_link': 'multimidia.curitiba.pr.gov.br/dadosabertos/OficinaMusica/2018-03-01_Oficina_de_Musica_-_Categoria_Nucleo_-_Base_de_Dados.csv',
  'database_date': datetime.date(2018, 3, 1)},
 {'database_id': '3980d747bc21fca2e6b89276189cedd0',
  'database_title': 'Oficina de Musica - Curso - Base de Dados',
  'database_link': 'multimidia.curitiba.pr.gov.br/dadosabertos/OficinaMusica/2018-03-01_Oficina_de_Musica_-_Curso_-_Base_de_Dados.csv',
  'database_date': datetime.date(2018, 3, 1)},
 {'database_id': 'd05dbc3b7fe240cbcba64dab1bc98ff7',
  'database_title': 'Oficina de Musica - Espaco - Base de Dados',
  'database_link': 'multimidia.curitiba.pr.gov.br/dadosabertos/OficinaMusica/2018-03-01_Oficina_de_Musica_-_Espaco_-_Base_de_Dados.csv',
  'database_date': datetime.date(2018, 3, 1)},
 {'database_id': 'fc8c0be0b3306e1f8999892cc009ff58',
  'database_title': 

In [58]:
test = pd.read_csv('http://multimidia.curitiba.pr.gov.br/dadosabertos/OficinaMusica/2018-03-01_Oficina_de_Musica_-_Categoria_Nucleo_-_Base_de_Dados.csv', encoding="ISO-8859-1", delimiter=';')

In [59]:
test.fillna('').head()

,CNU_IDF,CNU_TITULO,CNU_SITUACAO
0,-------,----------,------------
